<a href="https://colab.research.google.com/github/bramtechtalk/GA4-API-Data-2-Excel/blob/main/GA4_API_Data2Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Official documentation: https://developers.google.com/analytics/devguides/reporting/data/v1/quickstart-client-libraries#python
# Options, Metrics and Dimensions: https://developers.google.com/analytics/devguides/reporting/data/v1/api-schema

# Install module first
!pip install google-analytics-data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import modules

from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
)
from google.oauth2 import service_account
from google.protobuf.json_format import MessageToDict
import pandas as pd
import os
from google.colab import drive

In [ ]:
# Mount Google drive and list content

drive.mount('/content/drive')
os.listdir('./drive/MyDrive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['GA4-DATA', 'Colab Notebooks']

In [ ]:
# List content of the folder I created on Google Drive for this project

os.listdir('./drive/MyDrive/GA4-DATA')

['ga4-extract-data-388610-96abbf0fa466.json', 'GA4_Excel.xlsx']

In [ ]:
# Set credentials and define the right GA4 property

credentials = service_account.Credentials.from_service_account_file('./drive/MyDrive/GA4-DATA/ga4-extract-data-388610-96abbf0fa466.json') 
client = BetaAnalyticsDataClient(credentials=credentials)
property_id="376319614"

In [ ]:
# Get the data from the API

request = RunReportRequest(
    property=f"properties/{property_id}",
    dimensions=[Dimension(name="city"), Dimension(name="country")],
    metrics=[Metric(name="activeUsers"), Metric(name="sessions")],
    date_ranges=[DateRange(start_date="2020-03-31", end_date="today")],
    )

response = client.run_report(request)
response

dimension_headers {
  name: "city"
}
dimension_headers {
  name: "country"
}
metric_headers {
  name: "activeUsers"
  type_: TYPE_INTEGER
}
metric_headers {
  name: "sessions"
  type_: TYPE_INTEGER
}
rows {
  dimension_values {
    value: "Swalmen"
  }
  dimension_values {
    value: "Netherlands"
  }
  metric_values {
    value: "4"
  }
  metric_values {
    value: "6"
  }
}
rows {
  dimension_values {
    value: "Roermond"
  }
  dimension_values {
    value: "Netherlands"
  }
  metric_values {
    value: "2"
  }
  metric_values {
    value: "3"
  }
}
rows {
  dimension_values {
    value: "Brussels"
  }
  dimension_values {
    value: "Belgium"
  }
  metric_values {
    value: "1"
  }
  metric_values {
    value: "1"
  }
}
row_count: 3
metadata {
  currency_code: "EUR"
  time_zone: "Europe/Amsterdam"
}
kind: "analyticsData#runReport"

In [ ]:
# Turn the raw data into a Table

def ga4_result_to_df(response):
    """Original: print_run_report_response: Prints results of a runReport call. v2.1 changed by Bram to create DataFrame"""
    result_dict = {}  
    for dimensionHeader in response.dimension_headers:
        result_dict[dimensionHeader.name] = []
    for metricHeader in response.metric_headers:
        result_dict[metricHeader.name] = []
    for rowIdx, row in enumerate(response.rows):
        for i, dimension_value in enumerate(row.dimension_values):
            dimension_name = response.dimension_headers[i].name
            result_dict[dimension_name].append(dimension_value.value)
        for i, metric_value in enumerate(row.metric_values):
            metric_name = response.metric_headers[i].name
            result_dict[metric_name].append(metric_value.value)
    return pd.DataFrame(result_dict)

df = ga4_result_to_df(response)
df

,city,country,activeUsers,sessions
0,Swalmen,Netherlands,4,6
1,Roermond,Netherlands,2,3
2,Brussels,Belgium,1,1


In [ ]:
# Export the table to an Excel file on the Google Drive

df.to_excel('./drive/MyDrive/GA4-DATA/GA4_Excel.xlsx')